In [7]:
!pip install scrapy -q
import requests
import json 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
!pip install pgeocode -q
import pgeocode

# Scraping Autosphere 

In [ ]:
""""
marque_voiture = ['citroën']#,'peugeot','renault','toyota','volkswagen']

df = pd.DataFrame(columns = ['Marque', 'Modele', 'Transmission','Kilometre','Energie','Annee','Prix','Code_postal'])

for voiture in marque_voiture:
    print('PROCESSING _____ ', voiture, ' _____')
    headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
    r_autosphere = requests.get("https://www.autosphere.fr/recherche?chaine=" + voiture, headers = headers)
    soup = BeautifulSoup(r_autosphere.content)
    link=soup.select("div.bloc_infos_veh_parent")
    
    for l in link:
        Marque = l.select('span.marque')[0].get_text()
        Modele = l.select('span.modele')[0].get_text()
        Transmission = l.select('div.caract span')[-1].get_text()
        Kilometre = l.select('div.caract span')[2].get_text().replace(' ','').replace('km','')
        Energie = l.select('div.caract span')[0].get_text().replace(' / ','')
        Annee = l.select('div.caract span')[3].get_text().replace(' / ','').replace(' / ','')
        Prix = l.select('span.bloc_prix')[-1].get_text().replace('\n                                                                                        ','').replace('\xa0€\n                                        ','').replace('\n                                                        ','').replace('    ','').replace(' €','').replace('\n','')
        Code_postal = l.select('span.localisation.regular')[0].get_text()
        
        data = {
            "Marque":Marque,
            "Modele":Modele,
            #"Photo":Photo, 
            "Transmission":Transmission,
            "Kilometre":Energie,
            "Energie":Kilometre,
            "Annee":Annee,
            'Prix':Prix,
            'Code_postal':Code_postal
        }
        df = df.append(data, ignore_index = True)

print(df)
"""

In [ ]:
marque_voiture = ['citroën','peugeot','renault','toyota','volkswagen']
nombre_pages = [str(i) for i in range(2,91)]

df1 = pd.DataFrame(columns = ['Marque', 'Modele', 'Transmission','Kilometre','Energie','Annee','Prix','Code_postal'])

for voiture in marque_voiture:
    for pages in nombre_pages:
    
        #print('PROCESSING _____ ', voiture, ' _____')
        headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
        r_autosphere = requests.get("https://www.autosphere.fr/recherche?chaine={}&market=VO&page={}&critaire_checked[]=year&critaire_checked[]=discount&critaire_checked[]=emission_co2".format(voiture,pages), headers = headers)
        soup = BeautifulSoup(r_autosphere.content)
        link=soup.select("div.bloc_infos_veh_parent")
    
        for l in link:
            Marque = l.select('span.marque')[0].get_text()
            Modele = l.select('span.modele')[0].get_text()
            Transmission = l.select('div.caract span')[-1].get_text()
            Kilometre = l.select('div.caract span')[2].get_text().replace(' ','').replace('km','')
            Energie = l.select('div.caract span')[0].get_text().replace(' / ','')
            Annee = l.select('div.caract span')[3].get_text().replace(' / ','').replace(' / ','')
            Prix = l.select('span.bloc_prix')[-1].get_text().replace('\n                                                                                        ','').replace('\xa0€\n                                        ','').replace('\n                                                        ','').replace('    ','').replace(' €','').replace('\n','')
            Code_postal = l.select('span.localisation.regular')[0].get_text()
        
            data = {
                "Marque":Marque,
                "Modele":Modele,
                #"Photo":Photo, 
                "Transmission":Transmission,
                "Kilometre":Kilometre,
                "Energie":Energie,
                "Annee":Annee,
                'Prix':Prix,
                'Code_postal':Code_postal
            }
            df1 = df1.append(data, ignore_index = True)

df1.to_csv('autosphere.csv')

In [1]:
!pip install pgeocode -q
import pgeocode

# Récuperation des données gps

In [ ]:
lon_list = []
lat_list = []
for x in df1['Code_postal']:
    #print('PROCESSING _____ ', x , ' _____')
    nomi  =  pgeocode.Nominatim ('fr') 
    Geopy = nomi.query_postal_code(x)
    lon_list.append(Geopy['longitude'])
    lat_list.append(Geopy['latitude'])


df1['longitude'] = lon_list
df1['latitude'] = lat_list
df1.to_csv('autosphere.csv')

In [2]:
data = pd.read_csv('LaCentrale+autosphere+GPS+Scoring.csv')

In [8]:
data = pd.read_csv('LaCentrale+autosphere+GPS.csv')

In [9]:
data.columns

Index(['Unnamed: 0', 'Brand', 'Model', 'Gear', 'KM', 'Category', 'Energie',
       'Year', 'Price', 'Zip_code', 'longitude', 'latitude', 'City', 'State',
       'Region'],
      dtype='object')

In [10]:
data.drop('City', axis=1, inplace=True)
data.drop('Region', axis=1, inplace=True)
data.drop('Zip_code', axis=1, inplace=True)
del data['Unnamed: 0']

In [13]:
html = data.to_html()
print(html)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Fonctinon

In [14]:
#Crée une colonne scoring dans notre dataset
def scoring(data):
    scoring = 0
    
    if data['Gear'] == 'MANUELLE':
        scoring += 1
    elif data['Gear'] != 'MANUELLE':
        scoring += 2

    if data['KM'] <= 100:
        scoring += 10
    elif data['KM'] >100 <= 8000:# au debut la voiture decote plus vite 
        scoring += 9
    elif data['KM'] > 8000 <= 22893:
        scoring += 8
    elif data['KM'] > 22893 <= 51123:
        scoring += 7
    elif data['KM'] > 51123 <= 80000: # a plus de 80milles il y a les courois a changer entretient plus gros plus cher 
        scoring += 6 
    elif data['KM'] > 80000 <= 110000: 
        scoring += 5  
    elif data['KM'] > 110000 <= 130000: 
        scoring += 4
    elif data['KM'] > 130000 <= 150000: 
        scoring += 3
    elif data['KM'] > 150000 <= 200000: 
        scoring += 2
    elif data['KM'] > 200000 : 
        scoring += 1
   
    if data['Energie'] == 'ESSENCE' :
        scoring +=3
    elif data['Energie'] =='DIESEL':# 2024 plus de diesel a paris 
        scoring += 2
    elif data['Energie'] =='HYBRIDE':# norrme wltp = rejete un co2 inferieur a 90gr et 0gr pour l'electrique
        scoring += 4
    elif data['Energie'] == 'ELECTRIQUE':
        scoring += 5
    
    if data['Year'] < 2000 :
        scoring += 0
    elif data['Year'] >= 2000 <= 2013 :
        scoring += 1
    elif data['Year'] >= 2014 <= 2018 :
        scoring += 2
    elif data['Year'] > 2018 <= 2019 : # 2019 a 2020
        scoring += 3
    elif data['Year'] > 2019 <= 2020 :
        scoring += 4
    elif data['Year'] > 2020 :
        scoring += 5
        
    return scoring

data['scoring'] = data.apply(scoring, axis=1)
data

,Brand,Model,Gear,KM,Category,Energie,Year,Price,longitude,latitude,State,scoring
0,RENAULT,CLIO,MANUELLE,7577,Second-Hand,DIESEL,2020,17830.0,-0.510499,44.878583,NOUVELLE-AQUITAINE,13
1,RENAULT,CLIO,MANUELLE,49851,Second-Hand,DIESEL,2019,11540.0,0.369879,45.044040,NOUVELLE-AQUITAINE,13
2,RENAULT,CLIO,MANUELLE,19318,Second-Hand,ESSENCE,2020,13360.0,0.369879,45.044040,NOUVELLE-AQUITAINE,14
3,RENAULT,MEGANE,MANUELLE,124202,Second-Hand,DIESEL,2018,11530.0,3.121595,50.631513,HAUTS-DE-FRANCE,13
4,RENAULT,MEGANE,MANUELLE,66729,Second-Hand,DIESEL,2019,19360.0,3.122092,50.626434,HAUTS-DE-FRANCE,13
...,...,...,...,...,...,...,...,...,...,...,...,...
11573,VOLKSWAGEN,T-ROC,AUTOMATIQUE,69650,Second-Hand,DIESEL,2019,27990.0,3.121300,50.753300,HAUTS-DE-FRANCE,14
11574,VOLKSWAGEN,POLO,AUTOMATIQUE,29736,Second-Hand,DIESEL,2017,15987.0,6.149300,48.683700,GRAND EST,14
11575,VOLKSWAGEN,T-ROC,AUTOMATIQUE,55890,Second-Hand,ESSENCE,2018,25900.0,5.223900,43.416000,PROVENCE-ALPES-CÔTE D'AZUR,15
11576,VOLKSWAGEN,TIGUAN,MANUELLE,61873,Second-Hand,DIESEL,2018,21990.0,4.033300,49.250000,GRAND EST,13


In [75]:
#crée une fonction pour lancer une recherche dans notre base de données
def scoring():
    
    scoring = 0
    
    Question =input('Quel est votre transmission de preferance ? :')
    if Question == 'Manuelle':
        scoring += 1
    elif Question != 'Manuelle':
        scoring -= 1
    print(scoring)
    
    Question_2 = input ('Nombre de kilometre souhaité ? :')
    if Question_2 == 'pas beaucoup???':
        scoring += 1
    elif Question_2 == 'moyen':
        scoring += 1
    elif Question_2 == 'beaucoup':
        scoring  -= 1
    else  :
        scoring = 0
    print(scoring)
    
    question_3 = input ('quel type de carburant préférez vous ? :')
    if question_3 == 'Essance' :
        scoring -=1
    elif question_3 =='Diesel':
        scoring += 1
    elif question_3 =='Hybride':
        scoring += 1
    elif question_3 == 'Electrique':
        scoring += 1
    elif question_3 == 'GPL':
        scoring += 1
    elif question_3 == 'GNV' : 
        scoring -=1
    else :
        scoring = 0
    print(scoring)
    
    Question_4 = input ('Quel année de mise en circulation souhaitez vous ? :')
    if int(Question_4) <= 2013 :
        scoring += 1
    elif int(Question_4) >= 2014 <= 2018 :
        scoring += 1
    elif int(Question_4) > 2019 : # 2019 a 2020
        scoring += 1
    print(scoring)
    
    Question_5 = input ('Quel est votre budget pour votre achat ? :')
    if int(Question_5) <= 7000 :
        scoring += 1
    elif int(Question_5) > 7000 <= 14000 :
        scoring += 1
    elif int(Question_5) > 14000 <= 21000 :
        scoring += 1
    elif int(Question_5) > 21000 <= 21000 :
        scoring += 1
    elif int(Question_5) > 28000 <= 35000 : 
        scoring += 1
    elif int(Question_5) > 35000 <= 42000 :
        scoring += 1
    elif int(Question_5) > 42000 :
        scoring += 1
    else :
        scoring = 0
    print(scoring)
    
    return scoring

print('votre score est', scoring )

votre score est <function scoring at 0x7f0f0ca8b310>


In [17]:
def recherche():
    scoring = 0
    
    reponse = {'marque':[],'modele':[],'transmission':[], 'kilometre':[], 'carburant':[],'annee':[],'budget':[]}
    Question = input('Quel marque de voiture preferez vous ? :')
    #Question = model.predict()
    reponse['Marque'] = Question
    
    Question_1 = input('Quel modele de voiture recherchez vous ? :')
    reponse['modele'] = Question_1
    
    Question_2 =input('Quel est votre transmission de preferance ? :')
    reponse['transmission'] = Question_2
   
    Question_3 = input ('Nombre de kilometre souhaité ? :')
    reponse['kilometre'] = int(Question_3)
    
    question_4 = input ('quel type de carburant préférez vous ? :')
    reponse['carburant'] = question_4
    
    Question_5 = input ('Quel année de mise en circulation souhaitez vous ? :')
    reponse['annee'] = int(Question_5)
    
    Question_6 = input ('Quel est votre budget pour votre achat ? :')
    reponse['budget'] = (Question_6)
    
    
    return reponse

In [21]:
reponse = recherche()
mask = (data['Brand'] == reponse['Marque']) & (data['Model'] == reponse['modele']) &\
        (data['Gear'] == reponse['transmission']) &\
        (data['KM'] <= reponse['kilometre']) &\
        (data['Energie'] == reponse['carburant']) & (data['Year'] == reponse['annee']) & \
        (data['Price'] == reponse['budget'])
data.loc[mask]

Quel marque de voiture preferez vous ? : RENAULT
Quel modele de voiture recherchez vous ? : CLIO
Quel est votre transmission de preferance ? : MANUELLE
Nombre de kilometre souhaité ? : 7577
quel type de carburant préférez vous ? : DIESEL
Quel année de mise en circulation souhaitez vous ? : 2020
Quel est votre budget pour votre achat ? : 20000


,Brand,Model,Gear,KM,Category,Energie,Year,Price,longitude,latitude,State,scoring


In [90]:
int(reponse['kilometre']) == data.loc[0,'Kilometre']

True

In [8]:
#Crée une colonne pour le Price_recommendation

price_avg = data['Price'].mean()
price_std = data['Price'].std()
max_interval = price_avg+price_std
min_interval = price_avg-price_std
data['Price_recommendation'] = data['Price'].apply(lambda x: 'Expensive' if x > max_interval
                                                else 'In market' if max_interval >= x >= min_interval
                                                else 'Good price' if x < min_interval
                                                else x
                                                )

In [9]:
data


,Unnamed: 0,Brand,Model,Gear,KM,Energie,Year,Price,Zip_code,longitude,latitude,city,state,region,Category,Price_recommendation
0,0,RENAULT,CLIO,MANUELLE,7577,DIESEL,2020,17830.0,33310,-0.510499,44.878583,Lormont,Nouvelle-Aquitaine,Gironde,Second-Hand,In market
1,2,RENAULT,CLIO,MANUELLE,49851,DIESEL,2019,11540.0,24400,0.369879,45.044040,"Saint-Médard-de-Mussidan, Église-Neuve-d’Issac...",Nouvelle-Aquitaine,Dordogne,Second-Hand,In market
2,3,RENAULT,CLIO,MANUELLE,19318,ESSENCE,2020,13360.0,24400,0.369879,45.044040,"Saint-Médard-de-Mussidan, Église-Neuve-d’Issac...",Nouvelle-Aquitaine,Dordogne,Second-Hand,In market
3,4,RENAULT,KADJAR,MANUELLE,17010,DIESEL,2021,27280.0,31120,1.390001,43.519138,"Roques, Goyrans, Portet-sur-Garonne, Pinsaguel...",Occitanie,Haute-Garonne,Second-Hand,In market
4,5,RENAULT,MEGANE,MANUELLE,124202,DIESEL,2018,11530.0,59650,3.121595,50.631513,Villeneuve-d'Ascq,Hauts-de-France,Nord,Second-Hand,In market
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13347,18790,VOLKSWAGEN,GOLF SW,AUTOMATIQUE,5000,DIESEL,2021,28999.0,79000,-0.481267,46.323100,"Niort, Bessines, Sciecq",Nouvelle-Aquitaine,Deux-Sèvres,Second-Hand,Expensive
13348,18791,VOLKSWAGEN,T-ROC,AUTOMATIQUE,55890,ESSENCE,2018,25900.0,13700,5.223900,43.416000,"Marignane, Saint-Victoret",Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Second-Hand,In market
13349,18792,VOLKSWAGEN,GOLF SPORTSVAN,MANUELLE,106800,DIESEL,2015,13390.0,86000,0.333300,46.583300,Poitiers,Nouvelle-Aquitaine,Vienne,Second-Hand,In market
13350,18793,VOLKSWAGEN,TIGUAN,MANUELLE,61873,DIESEL,2018,21990.0,51100,4.033300,49.250000,Reims,Grand Est,Marne,Second-Hand,In market


In [ ]:
data.to_csv('LaCentrale+autosphere+GPS+Scoring.csv')